In [22]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pydicom
import os
from scipy.ndimage import zoom
import scipy
from scipy.ndimage import affine_transform

In [23]:
os.chdir("/Users/lindatang/Desktop/tumor_dl")

ax_file_name = os.listdir("test_folder/t1_ax")
sag_file_name = os.listdir("test_folder/t1_sag")
cor_file_name = os.listdir("test_folder/t1_cor")

In [24]:
def load_dicom_series(dicom_files, view):
    slices = [pydicom.dcmread(dcm) for dcm in dicom_files]
    if view =="cor":
        sort_dim = 1
    elif view == "ax":
        sort_dim = 2
    elif view =="sag":
        sort_dim = 2
    slices.sort(key=lambda x: float(x.ImagePositionPatient[sort_dim]))
    image = np.stack([s.pixel_array for s in slices], axis=-1)
    return image, slices[0].PixelSpacing, slices[0].ImageOrientationPatient, slices[0].ImagePositionPatient

In [25]:
os.chdir("test_folder/t1_ax")
data1, spacing1, orientation1, position1 = load_dicom_series(ax_file_name, view="ax") # 256
os.chdir("../t1_sag")
data2, spacing2, orientation2, position2 = load_dicom_series(sag_file_name, view="sag") # 320
os.chdir("../t1_cor")
cor_file_name = [x for x in cor_file_name if x != ".DS_Store"]
data3, spacing3, orientation3, position3 = load_dicom_series(cor_file_name, view="cor") # 256

In [26]:
target_shape = tuple([320, 320])

def resize_image(image, target_shape):
    zoom_factors = [t / float(s) for t, s in zip(target_shape, image.shape)]
    return zoom(image, zoom_factors, order=1)

resized_image = resize_image(data1[:, :, 0], target_shape)
#plt.imshow(resized_image, cmap = "gray")

# iterate to resize all images
data1_resized = []
for i in range(len(data1[1, 1, :])):
    resized_image = resize_image(data1[:, :, i], target_shape)
    data1_resized.append(resized_image)
    
data1_resized = np.stack(data1_resized, axis=-1)

# iterate to resize all images
data3_resized = []
for i in range(len(data3[1, 1, :])):
    resized_image = resize_image(data3[:, :, i], target_shape)
    data3_resized.append(resized_image)
    
data3_resized = np.stack(data3_resized, axis=-1)

In [27]:
def resample_data(data, target_slices):
    current_slices = data.shape[2]
    zoom_factor = target_slices / current_slices
    # Resample along the slice axis
    resampled_data = scipy.ndimage.zoom(data, (1, 1, zoom_factor), order=1)
    return resampled_data

data3_resized = resample_data(data3_resized, 35)

In [28]:
combined_data = np.stack((data1_resized, data2, data3_resized), axis=-1)
combined_data.shape

(320, 320, 35, 3)

In [29]:
affine = np.eye(4)
combined_img = nib.Nifti1Image(combined_data, affine=affine)
os.chdir("/Users/lindatang/Desktop/tumor_dl")
nib.save(combined_img, 'combined.nii')
